In [1]:
import pandas as pd

In [2]:
fatais = pd.read_excel('acidentes_fatais.xlsx')
nao_fatais = pd.read_csv('acidentes_naofatais.csv', encoding='unicode_escape', engine='python', sep=';')
acidentes_federais = pd.read_csv('accidents_data.csv', encoding='latin', sep=';')

c:\Users\99823045\projects\infosiga\.venv\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
def convert_to_str(value):
    if value:
        return str(int(value))
    else:
        return None
acidentes_federais['br'] = acidentes_federais['br'].fillna('').apply(convert_to_str)

In [4]:
fatais = fatais.query('Jurisdição != "FEDERAL"')
nao_fatais = nao_fatais.query('Jurisdição != "FEDERAL"')

In [5]:
fatais = fatais.query('`Lat (GEO)` != "NAO DISPONIVEL" or `Long (GEO)` != "NAO DISPONIVEL"')
nao_fatais = nao_fatais.query('`LAT_(GEO)` != "NAO DISPONIVEL" or `LONG_(GEO)` != "NAO DISPONIVEL"')

In [6]:
fatais = fatais[fatais['Data do Acidente'].dt.year >= 2017]
nao_fatais = nao_fatais[pd.to_datetime(nao_fatais['Data do Acidente']).dt.year >= 2017]

In [7]:
nao_fatais = nao_fatais.applymap(lambda x: None if x == 'NAO DISPONIVEL' else x)
fatais = fatais.applymap(lambda x: None if x == 'NAO DISPONIVEL' else x)
nao_fatais['Hora do Acidente'] = nao_fatais['Hora do Acidente'].apply(lambda x: x + ':00' if x is not None else x)
fatais['Hora do Acidente'] = fatais['Hora do Acidente'].apply(lambda x: x + ':00' if x is not None else x)

In [8]:
keep_cols1 = ['Data do Acidente', 'Dia da Semana', 'Hora do Acidente', 'uf', 'Logradouro',
              'Numero/KM', 'Município', 'causa_acidente', 'tipo_acidente', 'Classificação',
              'Turno', 'sentido_via', 'Condições Climáticas', 'Tipo de pista', 'Traçado',
              'Pessoas', 'mortos', 'Pessoas Envolvidas - Grave', 'LAT_(GEO)', 'LONG_(GEO)']

pessoas_cols = ['Pessoas Envolvidas - Grave', 'Pessoas Envolvidas - Ileso',
                'Pessoas Envolvidas - Leve']

nao_fatais = nao_fatais[(nao_fatais[pessoas_cols] > 0).any(axis=1)].copy()

nao_fatais['Pessoas'] = nao_fatais[pessoas_cols].sum(axis=1)
nao_fatais['causa_acidente'] = None
nao_fatais['tipo_acidente'] = None
nao_fatais['sentido_via'] = None
nao_fatais['mortos'] = 0
nao_fatais['uf'] = 'SP'

In [9]:
def classification(col):
    if col > 0:
        return 'Com Vítimas Feridas'
    else:
        return 'Sem Vítimas'

nao_fatais['Classificação'] = (nao_fatais[['Pessoas Envolvidas - Grave',
                                          'Pessoas Envolvidas - Leve']]
                               .sum(axis=1).apply(classification))

In [10]:
rename_cols1 = dict(zip(keep_cols1, list(acidentes_federais.columns)))
nao_fatais = nao_fatais[keep_cols1].rename(columns=rename_cols1)

In [11]:
keep_cols2 = ['Data do Acidente', 'Dia da semana', 'Hora do Acidente', 'uf', 
              'Logradouro', 'Numeral / KM', 'Município', 'causa_acidente',
              'tipo_acidente', 'Classificação', 'Turno', 'sentido_via',
              'Condições Climáticas (SIOPM)', 'Tipo de pista (SIOPM)', 'Traçado',
              'Pessoas', 'Quantidade de vítimas', 'feridos_graves', 'Lat (GEO)', 'Long (GEO)']

fatais['Classificação'] = 'Com Vítimas Fatais'
fatais['uf'] = 'SP'
fatais['sentido_via'] = None
fatais['causa_acidente'] = None
fatais['tipo_acidente'] = None
fatais['Traçado'] = None
fatais['Pessoas'] = fatais['Quantidade de vítimas']
fatais['feridos_graves'] = 0

In [12]:
rename_cols2 = dict(zip(keep_cols2, list(acidentes_federais.columns)))
fatais = fatais[keep_cols2].rename(columns=rename_cols2)
fatais['pessoas'] = fatais['pessoas'].astype('int64')
fatais['mortos'] = fatais['mortos'].astype('int64')

In [13]:
fatais['data_inversa'] = fatais['data_inversa'].dt.date.astype('object')

In [15]:
final_acidentes = pd.concat([acidentes_federais, nao_fatais, fatais]).reset_index(drop=True)

In [16]:
final_acidentes.to_csv('acidentes_final.csv', index=False, encoding='latin1', sep=';')